In [1]:
!pip install keras

## Imports

In [2]:
import numpy as np
import scipy as sp

import tensorflow as tf

import heapq
import math

## Load Dataset

In [3]:
def load_rating_file_as_list(filename):
    rating_list = []
    
    with open(filename, "r") as f:
        line = f.readline()
        
        while line and line != "":
            arr = line.split("\t")
            user, item = int(arr[0]), int(arr[1])
            rating_list.append([user, item])
            line = f.readline()
    
    return rating_list

def load_negative_file(filename):
    negative_list = []
    
    with open(filename, "r") as f:
        line = f.readline()
        
        while line and line != "":
            arr = line.split("\t")
            negatives = []
            
            for x in arr[1:]:
                negatives.append(int(x))
            
            negative_list.append(negatives)
            
            line = f.readline()
    
    return negative_list

def load_rating_file_as_matrix(filename):
    num_users, num_items = 0, 0
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            u, i = int(arr[0]), int(arr[1])
            num_users = max(num_users, u)
            num_items = max(num_items, i)
            line = f.readline()
    
    mat = sp.sparse.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            if (rating > 0):
                mat[user, item] = 1.0
            line = f.readline()    
    return mat

In [5]:
train = load_rating_file_as_matrix('./Data/ml-1m/ml-1m.train.rating')
test_ratings = load_rating_file_as_list('./Data/ml-1m/ml-1m.test.rating')
test_negatives = load_negative_file('./Data/ml-1m/ml-1m.test.negative')

In [6]:
num_users, num_items = train.shape
print('Loaded Data. # Users:', num_users, '# Items:', num_items, '# Train:', train.nnz, '# Test:', len(test_ratings))

Loaded Data. # Users: 6040 # Items: 3706 # Train: 994169 # Test: 6040


### Helper to Generate Negative Training Examples

In [7]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while train.get((u, j)):
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

## Create Model

In [8]:
from keras.models import Model
from keras.layers import Embedding, Input, Dense, Reshape, Multiply, Flatten, Lambda, Concatenate, Conv2D, MaxPool2D
from keras import initializers, regularizers
import sys

def get_OuterProductmodel(num_users, num_items, latent_dim):
    user_input = Input(shape = (1,), dtype = 'int32', name = 'user')
    item_input = Input(shape = (1,), dtype = 'int32', name = 'item')

    user_embedding = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)
    item_embedding = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)

    user_latent = Flatten()(user_embedding(user_input))
    item_latent = Flatten()(item_embedding(item_input))

    latent_map = tf.linalg.matmul(tf.expand_dims(user_latent, -1), tf.expand_dims(item_latent, 1))

    x = tf.expand_dims(latent_map, -1)

    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = Flatten()(x)
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(x)

    return Model(inputs=[user_input, item_input], outputs=prediction)

In [14]:
# https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba
def relu_bn(inputs):
    relu = tf.keras.layers.ReLU()(inputs)
    bn = tf.keras.layers.BatchNormalization()(relu)
    return bn

def residual_block(x, downsample = False, filters = 16, kernel_size = 3):
    y = tf.keras.layers.Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = tf.keras.layers.Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = tf.keras.layers.Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = tf.keras.layers.Add()([x, y])
    out = relu_bn(out)
    return out

def get_ResidualModel(num_users, num_items, embedding_dim, num_filters):
    user_input = Input(shape = (1,), dtype = 'int32', name = 'user')
    item_input = Input(shape = (1,), dtype = 'int32', name = 'item')

    user_embedding = Embedding(input_dim = num_users, output_dim = embedding_dim, name = 'user_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)
    item_embedding = Embedding(input_dim = num_items, output_dim = embedding_dim, name = 'item_embed',
                             embeddings_initializer = initializers.RandomNormal(stddev = 0.01), 
                             embeddings_regularizer = regularizers.l2(0), input_length = 1)

    user_latent = Flatten()(user_embedding(user_input))
    item_latent = Flatten()(item_embedding(item_input))

    latent_map = tf.linalg.matmul(tf.expand_dims(user_latent, -1), tf.expand_dims(item_latent, 1))

    x = tf.expand_dims(latent_map, -1)
    
    layers = [2, 2, 2]
    
    for i, num_layers in enumerate(layers):
        for _ in range(num_layers):
            x = residual_block(x, downsample = False, filters = num_filters, kernel_size = 3)
          
        if i == len(layers) - 1:
            x = residual_block(x, downsample = False, filters = num_filters, kernel_size = 3)
        else:
            x = residual_block(x, downsample = True, filters = num_filters, kernel_size = 3)
    
    x = Flatten()(x)
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(x)
    
    return Model(inputs=[user_input, item_input], outputs=prediction)

In [25]:
from keras.optimizers import Adam

topK = 10

#model = get_OuterProductmodel(num_users, num_items, 64)
model = get_ResidualModel(num_users, num_items, embedding_dim = 4, num_filters = 32)
model.compile(optimizer=Adam(0.005), loss='binary_crossentropy')
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
item (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embed (Embedding)          (None, 1, 4)         24160       user[0][0]                       
__________________________________________________________________________________________________
item_embed (Embedding)          (None, 1, 4)         14824       item[0][0]                       
____________________________________________________________________________________________

## Define Evaluation Functions

In [26]:
def evaluateNDCG(ranked_list, target_item):
    for i in range(len(ranked_list)):
        if ranked_list[i] == target_item:
            return math.log(2) / math.log(i + 2)
  
    return 0

def hitRate(ranked_list, target_item):
    for rank in ranked_list:
        if target_item == rank:
            return 1
    return 0

# This method calculates all the evaluation metrics. Individual methods are called from here.
def evaluate(model, testPosRatings, testNegRatings, N):
    hits = []
    ndcgs = []
    for i in range(len(testPosRatings)):
        hit, ncdg = evaluate_one(model, testPosRatings[i], testNegRatings[i], N)
        hits.append(hit)
        ndcgs.append(ncdg)
        
    return np.array(hits).mean(), np.array(ndcgs).mean()

def evaluate_one(model, posRating, negRatings, N):
    user = posRating[0]
    movie = posRating[1]
    negRatings.append(movie)

    user_input = np.full(len(negRatings), user)

    predictions = model.predict([user_input, np.array(negRatings)], batch_size = 100)

  # associate item with predictions
    items = {}
    for i in range(len(predictions)):
        items[negRatings[i]] = predictions[i]
    negRatings.pop()

    rankedList = heapq.nlargest(N, items, items.get)
    ndcg = evaluateNDCG(rankedList, movie)
    hit = hitRate(rankedList, movie)

    return hit, ndcg

## Train and Save Best Model

In [27]:
NUM_EPOCHS = 20
best_hr = 0
best_ncdg = 0
best_epoch = -1
model_path = "ResNet_Model.h5"

user_input, item_input, labels = get_train_instances(train, num_negatives = 4)
model.evaluate([np.array(user_input), np.array(item_input)],
                      np.array(labels),
                      batch_size = 256)

hit_rate, ncdg = evaluate(model, test_ratings, test_negatives, N = 10)
print('Initial Model', 'Hit Rate:', hit_rate, 'NCDG:', ncdg)

for epoch in range(1, NUM_EPOCHS + 1):
    user_input, item_input, labels = get_train_instances(train, num_negatives = 4)

    hist = model.fit([np.array(user_input), np.array(item_input)],
                      np.array(labels),
                      batch_size = 512, epochs = 1)

    hit_rate, ncdg = evaluate(model, test_ratings, test_negatives, N = 10)
    print('Epoch', epoch, 'Hit Rate:', hit_rate, 'NCDG:', ncdg)
    model.save(model_path)

    if hit_rate > best_hr:
        best_hr, best_ncdg, best_iter = hit_rate, ncdg, epoch
        model.save(model_path, overwrite=True)

print("Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ncdg))
print("The best Outer Product model is saved to %s" %(model_path))

19418/19418 [==============================] - 73s 4ms/step - loss: 0.6932
Initial Model Hit Rate: 0.11357615894039735 NCDG: 0.05362733732335355
9709/9709 [==============================] - 99s 10ms/step - loss: 0.3760
Epoch 1 Hit Rate: 0.44519867549668873 NCDG: 0.24532661019846647
9709/9709 [==============================] - 97s 10ms/step - loss: 0.3487
Epoch 2 Hit Rate: 0.48443708609271524 NCDG: 0.26758745588254657
9709/9709 [==============================] - 96s 10ms/step - loss: 0.3340
Epoch 3 Hit Rate: 0.5016556291390728 NCDG: 0.2777408693810437
9709/9709 [==============================] - 96s 10ms/step - loss: 0.3273
Epoch 4 Hit Rate: 0.5231788079470199 NCDG: 0.28444157653746105
9709/9709 [==============================] - 97s 10ms/step - loss: 0.3205
Epoch 5 Hit Rate: 0.5384105960264901 NCDG: 0.2963511111263239
9709/9709 [==============================] - 96s 10ms/step - loss: 0.3149
Epoch 6 Hit Rate: 0.5473509933774835 NCDG: 0.30252841078746157
9709/9709 [======================